In [ ]:
!pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import torch
import numpy as np

In [ ]:
dataset = load_dataset("emotion")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
# Check the unique labels in the dataset
unique_labels = dataset['train'].features['label'].names
num_labels = len(unique_labels)

print(f"Unique labels: {unique_labels}")
print(f"Number of labels: {num_labels}")

Unique labels: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
Number of labels: 6


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize the dataset
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split the dataset into train and test sets
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import torch
import numpy as np

# Load the dataset
dataset = load_dataset("emotion")

# Tokenize the dataset
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Use a subset of the training dataset
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(3000))  # Using only 500 samples
test_dataset = tokenized_datasets['test']

# Load the pre-trained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # Reduced epochs
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
)

# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = load_metric("accuracy")
    return metric.compute(predictions=predictions, references=labels)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.698293,0.751000
2,No log,0.355336,0.888000
3,0.675100,0.301202,0.899000
4,0.675100,0.302776,0.903500
5,0.675100,0.317325,0.907500


TrainOutput(global_step=940, training_loss=0.4094144293602477, metrics={'train_runtime': 476.2572, 'train_samples_per_second': 31.496, 'train_steps_per_second': 1.974, 'total_flos': 3946807572480000.0, 'train_loss': 0.4094144293602477, 'epoch': 5.0})

In [ ]:
import pickle

# Define the mapping from label IDs to emotion names
id2label = {
    0: "sad",
    1: "happy",
    2: "love",
    3: "angry",
    4: "fear",
    5: "surprise"
}

# Update the model's config with this mapping
model.config.id2label = id2label

# Move the model to the GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define the prediction function
def predict_emotions(text):
    sentences = text.split('.')
    results = []
    for sentence in sentences:
        if sentence.strip():  # Ignore empty sentences
            inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class_id = logits.argmax().item()
            predicted_emotion = model.config.id2label[predicted_class_id]
            results.append((sentence.strip(), predicted_emotion))
    return results

# Example text
text = "I am happy today. The weather is gloomy. why are you irritating me."

# Predict emotions
predicted_emotions = predict_emotions(text)

# Highlight sentences with their emotions
for sentence, emotion in predicted_emotions:
    print(f"{sentence}: {emotion}")

# Save the model's config and state_dict
model_info = {
    'config': model.config,
    'state_dict': model.state_dict()
}

with open('saved_model.pkl', 'wb') as f:
    pickle.dump(model_info, f)


I am happy today: happy
The weather is gloomy: sad
why are you irritating me: angry


In [ ]:
import pickle

# Define the mapping from label IDs to emotion names
id2label = {
    0: "sad",
    1: "happy",
    2: "love",
    3: "angry",
    4: "fear",
    5: "surprise"
}

# Update the model's config with this mapping
model.config.id2label = id2label

# Move the model to the GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define the prediction function
def predict_emotions(text):
    sentences = text.split('.')
    results = []
    for sentence in sentences:
        if sentence.strip():  # Ignore empty sentences
            inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class_id = logits.argmax().item()
            predicted_emotion = model.config.id2label[predicted_class_id]
            results.append((sentence.strip(), predicted_emotion))
    return results

# Example text
text = "Through turbulent skies and fleeting moments, a silent dance of shadows and whispers weaves a tapestry of human experience, embracing the ebb and flow of life's mysteries."

# Predict emotions
predicted_emotions = predict_emotions(text)

# Highlight sentences with their emotions
for sentence, emotion in predicted_emotions:
    print(f"{sentence}: {emotion}")




Through turbulent skies and fleeting moments, a silent dance of shadows and whispers weaves a tapestry of human experience, embracing the ebb and flow of life's mysteries: sad


In [ ]:
import pickle

# Specify the path to your .pkl file
pkl_file_path = '/content/saved_model.pkl'

# Load the model from the .pkl file
with open(pkl_file_path, 'rb') as f:
    loaded_model = pickle.load(f)

# Check the type of the loaded object
print(type(loaded_model))

# If the loaded object is not the model itself, investigate the loading process


<class 'dict'>


In [ ]:
import torch
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model info from the .pkl file
with open('/content/saved_model.pkl', 'rb') as f:
    model_info = pickle.load(f)

# Initialize the model with the saved configuration
model = AutoModelForSequenceClassification.from_config(model_info['config'])
model.load_state_dict(model_info['state_dict'])

# Define the tokenizer and model name
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Now you can use the loaded model for predictions


In [ ]:
# Example text for prediction
input_text = " One of my friends also worked on turning on and off a particular lamp using a trigger word kind of as a fun project. But what I want to show you is how you can build a trigger word detection system. The literature on trigger detection algorithm is still evolving so there isn't wide consensus yet on what's the best algorithm for trigger word detection."

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Perform inference using the loaded model
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class
predicted_class_id = torch.argmax(outputs.logits[0]).item()

# Map the predicted class ID to the corresponding emotion
predicted_emotion = model.config.id2label[predicted_class_id]

# Display the predicted emotion
print("Predicted emotion:", predicted_emotion)


Predicted emotion: happy


In [ ]:
import torch
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model info from the .pkl file
with open('/content/saved_model.pkl', 'rb') as f:
    model_info = pickle.load(f)

# Initialize the model with the saved configuration
model = AutoModelForSequenceClassification.from_config(model_info['config'])
model.load_state_dict(model_info['state_dict'])

# Define the tokenizer and model name
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define the mapping from label IDs to emotion names
id2label = {
    0: "sad",
    1: "happy",
    2: "love",
    3: "angry",
    4: "fear",
    5: "surprise"
}

# Update the model's config with this mapping
model.config.id2label = id2label

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the prediction function
def predict_emotions(text):
    sentences = text.split('.')
    results = []
    for sentence in sentences:
        if sentence.strip():  # Ignore empty sentences
            inputs = tokenizer(sentence.strip(), return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
            logits = outputs.logits
            predicted_class_id = logits.argmax().item()
            predicted_emotion = model.config.id2label[predicted_class_id]
            results.append((sentence.strip(), predicted_emotion))
    return results

# Example text
text = "As the sun dipped below the horizon, casting a warm glow over the city, I couldn't help but feel a mix of emotions. Walking through the bustling streets, I noticed couples strolling hand in hand, their eyes filled with affection and love, which brought a sense of warmth to the cool evening air. However, amidst the laughter and chatter, a sudden loud argument erupted, sending ripples of tension and unease through the crowd. The flickering streetlights and looming shadows added a touch of fear, heightening the senses as night descended. Just then, a sudden burst of energy and excitement filled the sky as fireworks exploded in a dazzling display, leaving everyone in awe and surprise."

# Predict emotions
predicted_emotions = predict_emotions(text)

# Highlight sentences with their emotions
for sentence, emotion in predicted_emotions:
    print(f"{sentence}: {emotion}")


As the sun dipped below the horizon, casting a warm glow over the city, I couldn't help but feel a mix of emotions: love
Walking through the bustling streets, I noticed couples strolling hand in hand, their eyes filled with affection and love, which brought a sense of warmth to the cool evening air: love
However, amidst the laughter and chatter, a sudden loud argument erupted, sending ripples of tension and unease through the crowd: angry
The flickering streetlights and looming shadows added a touch of fear, heightening the senses as night descended: fear
Just then, a sudden burst of energy and excitement filled the sky as fireworks exploded in a dazzling display, leaving everyone in awe and surprise: happy


In [ ]:
texts = [
"i feel happy"]

# Predict emotions
predicted_emotions = predict_emotions(texts)

# Highlight sentences with their emotions
for sentence, emotion in predicted_emotions:
    print(f"{sentence}: {emotion}")


i feel happy: surprise


In [ ]:
import pickle

# Define the mapping from label IDs to emotion names
id2label = {
    0: "sad",
    1: "happy",
    2: "love",
    3: "angry",
    4: "fear",
    5: "surprise"
}

# Assuming 'model' and 'tokenizer' are already defined and trained

# Update the model's config with this mapping
model.config.id2label = id2label

# Prepare the model information to save
model_info = {
    'config': model.config,
    'state_dict': model.state_dict()
}

# Define the path to save the model in Google Drive
model_path = '/content/drive/MyDrive/certificates/saved_model.pkl'

# Save the model's config and state_dict to the specified path
with open(model_path, 'wb') as f:
    pickle.dump(model_info, f)

print(f"Model saved to {model_path}")


Model saved to /content/drive/MyDrive/certificates/saved_model.pkl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the path to the saved model in Google Drive
model_path = '/content/drive/MyDrive/certificates/saved_model.pkl'

# Load the saved model info from the .pkl file
with open(model_path, 'rb') as f:
    model_info = pickle.load(f)

# Initialize the model with the saved configuration
model = AutoModelForSequenceClassification.from_config(model_info['config'])
model.load_state_dict(model_info['state_dict'])

# Move the model to the GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define the tokenizer and model name
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Update the model's config with id2label mapping if not already present
id2label = {
    0: "sad",
    1: "happy",
    2: "love",
    3: "angry",
    4: "fear",
    5: "surprise"
}
model.config.id2label = id2label

# Define the prediction function
def predict_emotions(text):
    sentences = text.split('.')
    results = []
    for sentence in sentences:
        if sentence.strip():  # Ignore empty sentences
            inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                predicted_class_id = logits.argmax().item()
                predicted_emotion = model.config.id2label[predicted_class_id]
                results.append((sentence.strip(), predicted_emotion))
    return results

# Example text
# Example text for prediction
text = "Under the gloomy skies, the family gathered to celebrate the graduation of their eldest son, who had worked tirelessly for years. Amidst the chatter and laughter, the grandmother's eyes sparkled with pride as she reminisced about the hard times they had endured. The father, feeling a surge of affection, gave his son a rare hug, which took the son by surprise but filled him with warmth. Meanwhile, in the corner, a heated argument erupted between two uncles over an old family dispute, their voices rising in anger, causing the younger children to watch with a mix of fear and curiosity. Suddenly, a loud crash from the kitchen startled everyone, only to reveal the cat had knocked over a vase, leaving the mother dismayed at the mess but relieved it was nothing serious. As the evening drew to a close, the family sat together, enjoying the calm after the storm, feeling the bittersweet blend of love, joy, and the underlying tensions that bound them together."

# Predict emotions
predicted_emotions = predict_emotions(text)

# Highlight sentences with their emotions
for sentence, emotion in predicted_emotions:
    print(f"{sentence}: {emotion}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Under the gloomy skies, the family gathered to celebrate the graduation of their eldest son, who had worked tirelessly for years: sad
Amidst the chatter and laughter, the grandmother's eyes sparkled with pride as she reminisced about the hard times they had endured: happy
The father, feeling a surge of affection, gave his son a rare hug, which took the son by surprise but filled him with warmth: love
Meanwhile, in the corner, a heated argument erupted between two uncles over an old family dispute, their voices rising in anger, causing the younger children to watch with a mix of fear and curiosity: angry
Suddenly, a loud crash from the kitchen startled everyone, only to reveal the cat had knocked over a vase, leaving the mother dismayed at the mess but relieved it was nothing serious: fear
As the evening drew to a close, the family sat together, enjoying the c